In [ ]:
import numpy as np
import time
import datetime
import rsa_decoder
import sqlite3

connection = sqlite3.connect('shows.db', check_same_thread=False)
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS Shows
              (data_now  DATETIME,
                leakage_sensor_1 FLOAT,
                smoke_sensor_1 FLOAT,
                gas_leak_sensor FLOAT,
                temp_sensor FLOAT,
                humidity_sensor FLOAT,
                motion_sensor_1 FLOAT,
                opening_sensor_1 FLOAT)''')
cursor.execute("DELETE FROM Shows")

data_now = []

def rescale(value, old_min, old_max, new_min, new_max):
    rescaled = (((value - old_min) * (new_max - new_min)) / (old_max - old_min)) + new_min
    return rescaled

rsa_decoder.generate_keys()

# (key_pub, key_priv) = rsa.newkeys(512)

while 1:
    time.sleep(5)
    # датчик контроля протечек
    leakage_sensor = np.random.binomial(10, 0.5)
    leakage_sensor = rescale(leakage_sensor, 0, 10, 0, 1)
    leakage_sensor_1 = round(leakage_sensor)
    # датчик задымления !!!
    smoke_sensor = np.random.gumbel(0, 1)
    smoke_sensor = rescale(smoke_sensor, -5, 6, 0, 1)
    smoke_sensor_1 = round(smoke_sensor)
    # датчик утечки газа
    gas_leak_sensor = np.random.laplace(loc=0.0, scale=1.0, size=None)
    gas_leak_sensor = rescale(gas_leak_sensor, -5, 5, 0, 50)
    # датчик температуры
    temp_sensor = np.random.power(0.7)
    temp_sensor = rescale(temp_sensor, 0, 1, 0, 50)
    # датчик влажности воздуха
    humidity_sensor = np.random.uniform(0, 100)
    # датчик движения
    motion_sensor = np.random.weibull(2)
    motion_sensor = rescale(motion_sensor, 0, 3, 0, 1)
    motion_sensor_1 = round(motion_sensor)
    # датчик открытия окон и дверей
    opening_sensor = np.random.normal(100, 10)
    opening_sensor = rescale(opening_sensor, 70, 120, 0, 1)
    opening_sensor_1 = round(opening_sensor)

    data_now = datetime.datetime.now()
#     moreShows = [(data_now,
#                 rsa.encrypt((str(leakage_sensor_1)).encode('utf8'), key_pub),
#                 rsa.encrypt((str(smoke_sensor_1)).encode('utf8'), key_pub),
#                 rsa.encrypt((str(gas_leak_sensor)).encode('utf8'), key_pub),
#                 rsa.encrypt((str(temp_sensor)).encode('utf8'), key_pub),
#                 rsa.encrypt((str(humidity_sensor)).encode('utf8'), key_pub),
#                 rsa.encrypt((str(motion_sensor_1)).encode('utf8'), key_pub),
#                 rsa.encrypt((str(opening_sensor_1)).encode('utf8'), key_pub))]
#     moreShows = [(data_now,
#                 leakage_sensor_1,
#                 smoke_sensor_1,
#                 gas_leak_sensor,
#                 temp_sensor,
#                 humidity_sensor,
#                 motion_sensor_1,
#                 opening_sensor_1)]
    moreShows = [(rsa_decoder.encode(data_now),
                rsa_decoder.encode(leakage_sensor_1),
                rsa_decoder.encode(smoke_sensor_1),
                rsa_decoder.encode(gas_leak_sensor),
                rsa_decoder.encode(temp_sensor),
                rsa_decoder.encode(humidity_sensor),
                rsa_decoder.encode(motion_sensor_1),
                rsa_decoder.encode(opening_sensor_1))]


    cursor.executemany("INSERT INTO Shows VALUES ( ?, ?, ?, ?, ?, ?, ?, ?)", moreShows)
    connection.commit()

    cursor.execute("SELECT * FROM Shows")
    results = cursor.fetchall()
    print(results)
    
#     
#     cursor.close()

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd

[(b'2\x83E\xd5H\xe7\xff\xb3\xca\xf22q\xf3_H\x98\xa0z\xaf{\xc0\xc3g_&,e\xac\x80\xf2\xeam\rEc{q\x96\x9e,\x89?\x89\x9d\xe2\xe7O?8s\xebS]}\xff\x97\xad\xb3\xd85\x03\x8b\xb4\xac', b'\x81Hy_\x10r`\xbc\xac\x1ao\x952\x12\xf1Dk\xc0\x1a \x9eK\xb7\xfe\xa0\xab^\xa40}\xad`\x15\xc4\x9bM\xe4\x12xK\xbb\x9b:M\x8ad\xb4\xbfPg\xb37K\xf9\xc9\xa2\xf1\x89\x07\xa1s\x1e\xdd:', b's\xe4\x05\xe8\xa6\x00l\x03c,\xd9Q\xa0\xea"\x85U\xd5\x96`\x83\xe7R9\xebc$\xae#HO]`\x93Dt\xf5\xd7\'\xb5\x18\xfex\x14\xf0\x0c\x1a\xb2\x8c\x10o\xc9\xcf\x9d_\x03U\xa7t\xf0\xa6\xe1\x1dX', b'\x1aa!\x125,\x84pS\x99\x84\xb6\x82H\xd4\xa7\xe9\xee\xb8\x99\x15/\x0e\x7f\xee\xab\x8d\xff)\xd4\x8e\xa8\xc5\x8e8\xea0!&\xbd!0s\x045\x8a<\x01s\x98\xc0\x8eT\xa1\x8f62n\x99\xb3wMO\xbf', b'0\xd0\xd4\xf0\xa0k\xa0\\\x0c\xc0^\x7ffu2X\x1epz\xd7\xd4\xfc%\xa8\xef\\\x93\xf6\x88\x92m\xfd\x13\x7f\x07\xc2r\t\x15O\xc8\xf6\x19\xad\x12\xe21\x80\xb5\r\xa0\x18\xe2qa\x15X\xd1\xd7;\xbet\xc8\xc3', b'y\x91\xadZ\x0c.\xf0\xd1\xc9\xe9D\x0e7\xae|\xdd\x9b\xd5\xfbL\xfa\x02\xcd\x9frMu\xd